![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Disponibilización de modelos

En este notebook aprenderá a guardar un modelo y a hacerlo disponible como una API con la librería Flask. Una API (interfaz de programación de aplicaciones) es un conjunto de definiciones y protocolos que permiten que servicios, en este caso modelos, retornen resultados y respuestas sin necesidad de saber cómo están implementados.

## Instrucciones Generales:

Este notebook esta compuesto por dos secciones. En la primera, usted deberá entrenar y guardar (exportar) un modelo de random forest para predecir si una URL es phishing (fraudulenta) o no. En la segunda parte, usará el modelo entrenado y lo disponibilizara usando la libreria *Flask*. En el siguente paper puede conocer más detalles de la base de datos que usaremos y del problema: *A. Correa Bahnsen, E. C. Bohorquez, S. Villegas, J. Vargas, and F. A. Gonzalez, “Classifying phishing urls using recurrent neural networks,” in Electronic Crime Research (eCrime), 2017 APWG Symposium on. IEEE, 2017, pp. 1–8*. <a href='https://albahnsen.files.wordpress.com/2018/05/classifying-phishing-urls-using-recurrent-neural-networks_cameraready.pdf'>LINK</a>
  
Para realizar la actividad, solo siga las indicaciones asociadas a cada celda del notebook. 


## Importar base de datos y librerías

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Importación librerías
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import joblib
import os
os.chdir('..')

In [6]:
# Carga de datos de archivos .csv
data = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/phishing.csv')
data.head()

,url,phishing
0,http://www.subalipack.com/contact/images/sampl...,1
1,http://fasc.maximecapellot-gypsyjazz-ensemble....,1
2,http://theotheragency.com/confirmer/confirmer-...,1
3,http://aaalandscaping.com/components/com_smart...,1
4,http://paypal.com.confirm-key-21107316126168.s...,1


## Codificar variables categóricas
Relizar preprocesamiento de texto (URLs) para crear variables predictoras:

In [7]:
# Creación de columnas binarias que indican si la URL contiene la palabra clave (keywords)
keywords = ['https', 'login', '.php', '.html', '@', 'sign']
for keyword in keywords:
    data['keyword_' + keyword] = data.url.str.contains(keyword).astype(int)

# Definición de la variable largo de la URL
data['lenght'] = data.url.str.len() - 2

# Definición de la variable largo del dominio de la URL
domain = data.url.str.split('/', expand=True).iloc[:, 2]
data['lenght_domain'] = domain.str.len()

# Definición de la variable binaria que indica si es IP
data['isIP'] = (domain.str.replace('.', '') * 1).str.isnumeric().astype(int)

# Definicón de la variable cuenta de 'com' en la URL
data['count_com'] = data.url.str.count('com')

data.head()

,url,phishing,keyword_https,keyword_login,keyword_.php,keyword_.html,keyword_@,keyword_sign,lenght,lenght_domain,isIP,count_com
0,http://www.subalipack.com/contact/images/sampl...,1,0,0,0,0,0,0,47,18,0,1
1,http://fasc.maximecapellot-gypsyjazz-ensemble....,1,0,0,0,0,0,0,73,41,0,0
2,http://theotheragency.com/confirmer/confirmer-...,1,0,0,0,0,0,0,92,18,0,1
3,http://aaalandscaping.com/components/com_smart...,1,0,0,0,0,0,0,172,18,0,3
4,http://paypal.com.confirm-key-21107316126168.s...,1,0,0,0,0,0,0,90,50,0,1


In [8]:
# Separación de variables predictoras (X) y variable de interes (y)
X = data.drop(['url', 'phishing'], axis=1)
y = data.phishing

## Entrenar y guardar el modelo

Calibracion de parametros

In [15]:
def best_parameters(estimator_range, feature_range, depth_range):
  estim_scores = []
  feat_scores = []
  depth_scores = []
  max_params = {
      'max_estimators': 0,
      'max_estimator_v': 0,
      'max_features': 0,
      'max_features_v': 0,
      'max_depth': 0,
      'max_depth_v': 0
  }

  # Modelo inicial
  clf = RandomForestClassifier(random_state=0)
  print('Modelo inicial:', pd.Series(cross_val_score(clf, X, y, cv=10)).describe())
  
  # Prueba de parámetros
  for max_depth in depth_range:
    for max_features in feature_range:
      for n_estimators in estimator_range:
        clf = RandomForestClassifier(n_estimators=n_estimators, max_features=max_features, max_depth=max_depth, random_state=1, n_jobs=-1)
        cv_score = cross_val_score(clf, X, y, cv=5, scoring='accuracy').mean()

        if cv_score > max_params['max_depth_v']:
          max_params['max_depth_v'] = cv_score
          max_params['max_depth'] = max_depth
          max_params['max_features_v'] = cv_score
          max_params['max_features'] = max_features
          max_params['max_estimator_v'] = cv_score
          max_params['max_estimators'] = n_estimators

        estim_scores.append((n_estimators, cv_score))
        feat_scores.append((max_features, cv_score))
        depth_scores.append((max_depth, cv_score))
  # Create figure and subplots
  # fig, axs = plt.subplots(1, 3, figsize=(15, 5))

  # # Subplot 1: Estimator
  # axs[0].plot(estimator_range, estim_scores)
  # axs[0].set_title("Estimator")
  # axs[0].set_xlabel('n_estimators')
  # axs[0].set_ylabel('Accuracy')
  # axs[0].grid()

  # # Subplot 2: Features
  # axs[1].plot(feature_range, feat_scores, label="Cosine")
  # axs[1].set_title("Features")
  # axs[1].set_xlabel("n_features")
  # axs[1].set_ylabel("Accuracy")
  # axs[1].grid()

  # # Subplot 3: Depth
  # axs[2].plot(depth_range, depth_scores, label="Cosine")
  # axs[2].set_title("Depth")
  # axs[2].set_xlabel("n_depth")
  # axs[2].set_ylabel("Accuracy")
  # axs[2].grid()

  # # Show the plot
  # plt.show()

  # Print the best params
  print('Mejores parámetros:', max_params)

  return estim_scores, feat_scores, depth_scores, max_params

# Define los rangos
feature_range = range(1, len(X.columns) + 1)
estimator_range = range(10, 310, 10)
depth_range = range(1, 21)

# Ejecuta la función
estim_scores, feat_scores, depth_scores, max_params = best_parameters(estimator_range, feature_range, depth_range)

Modelo inicial: count    10.000000
mean      0.803600
std       0.006906
min       0.790500
25%       0.801687
50%       0.805000
75%       0.808312
max       0.812500
dtype: float64
Mejores parámetros: {'max_estimators': 300, 'max_estimator_v': 0.8189, 'max_features': 2, 'max_features_v': 0.8189, 'max_depth': 19, 'max_depth_v': 0.8189}


In [20]:
# Definición de modelo de clasificación Random Forest
clf = RandomForestClassifier(n_jobs=-1, n_estimators=300, max_depth=19, max_features=2,random_state=1)
cross_val_score(clf, X, y, cv=10)

array([0.82375, 0.81675, 0.8265 , 0.8145 , 0.82075, 0.82575, 0.82075,
       0.8155 , 0.81575, 0.806  ])

In [17]:
# Entrenamiento del modelo de clasificación Random Forest
clf.fit(X, y)

RandomForestClassifier(max_depth=19, max_features=2, n_estimators=300,
                       n_jobs=-1)

In [19]:
# Exportar modelo a archivo binario .pkl
joblib.dump(clf, 'model_deployment/phishing_clf.pkl', compress=3)

FileNotFoundError: [Errno 2] No such file or directory: '../model_deployment/phishing_clf.pkl'

In [ ]:
# Importar modelo y predicción
from model_deployment.m09_model_deployment import predict_proba

# Predicción de probabilidad de que un link sea phishing
predict_proba('http://www.vipturismolondres.com/com.br/?atendimento=Cliente&/LgSgkszm64/B8aNzHa8Aj.php')

## Disponibilizar modelo con Flask

Para esta sección del notebook instale las siguientes librerías *!pip install flask* y *!pip install flask_restplus*.

In [ ]:
# Importación librerías
from flask import Flask
from flask_restx import Api, Resource, fields

In [ ]:
# Definición aplicación Flask
app = Flask(__name__)

# Definición API Flask
api = Api(
    app, 
    version='1.0', 
    title='Phishing Prediction API',
    description='Phishing Prediction API')

ns = api.namespace('predict', 
     description='Phishing Classifier')

# Definición argumentos o parámetros de la API
parser = api.parser()
parser.add_argument(
    'URL', 
    type=str, 
    required=True, 
    help='URL to be analyzed', 
    location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})

In [ ]:
# Definición de la clase para disponibilización
@ns.route('/')
class PhishingApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()
        
        return {
         "result": predict_proba(args['URL'])
        }, 200

In [ ]:
# Ejecución de la aplicación que disponibiliza el modelo de manera local en el puerto 5000
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

El modelo debe haber quedado disponibilizado en el puerto 5000. Para predecir la probabilidad de que una URL sea fraudulenta (phishing) copie en la barra de busqueda de su navegador la siguiente dirección (http://localhost:5000/predict/?URL=) y agregregue al final de esta la URL que desee precir. Por ejemplo, al copiar la URL http://localhost:5000/predict/?URL=http://consultoriojuridico.co/pp/www.paypal.com/, la API retornará la probabilidad de que la URL http://consultoriojuridico.co/pp/www.paypal.com/ sea phishing.